<p align="center"><img src="logo/spark.png" alt="Hadoop Logo" width="250"/></p>
# **Lab 2 - Part 3 - Spark**
#### This lab will demonstrate how to perform web server log analysis with Spark. Log data is a very large, common data source and contains a rich set of information.  It comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more. This lab will show you how to use Spark on real-world text-based production logs and fully harness the power of that data.

### **Part 1: Apache Web Server Log file format**


The log files that we use for this assignment are in the [Apache Common Log Format (CLF)](http://httpd.apache.org/docs/1.3/logs.html#common). The log file entries produced in CLF will look something like this:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
Each part of this log entry is described below:
* **`127.0.0.1`:** this is the IP address (or host name, if available) of the client (remote host) which made the request to the server. 
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available. 
* **`[01/Aug/1995:00:00:01 -0400]`:** the time that the server finished processing the request. The format is:
`[day/month/year:hour:minute:second timezone]`.
* **`"GET /images/launch-logo.gif HTTP/1.0"`:** this is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint, and the client protocol version. 
* **`200`:** this is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).
* **`1839`:** the last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).
 
Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log line. The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a Python `datetime` object using the given `parse_apache_time` function.

We, then, create the primary RDD and  we'll use in the rest of this assignment. We first load the text file and convert each line of the file into an element in an RDD. Next, we use `map(parseApacheLogLine)` to apply the parse function to each element and turn each line into a pair `Row` object. Finally, we cache the RDD in memory since we'll use it throughout this notebook. The log file is available at `data/apache/apache.log`.

In [87]:
// Just run this code
import scala.util.matching
import org.apache.spark.rdd.RDD

case class Cal(year: Int, month: Int, day: Int, hour: Int, minute: Int, second: Int)                

case class Row(host: String,
               client_identd: String,
               user_id: String,
               date_time: Cal,
               method: String,
               endpoint: String,
               protocol: String,
               response_code: Int,
               content_size: Long)
                

val month_map = Map("Jan" -> 1, "Feb" -> 2, 
                    "Mar" -> 3, "Apr" -> 4, 
                    "May" -> 5, "Jun" -> 6, 
                    "Jul" -> 7, "Aug" -> 8,  
                    "Sep" -> 9, "Oct" -> 10, 
                    "Nov" -> 11, "Dec" -> 12)

def parse_apache_time(s: String): Cal = {
    // Convert Apache time format into a Python datetime object
    // Args:
    //    s (str): date and time in Apache time format
    // Returns:
    //   datetime: datetime object (ignore timezone for now)

    return Cal(s.substring(7, 11).toInt,
            month_map(s.substring(3, 6)),
            s.substring(0, 2).toInt,
            s.substring(12, 14).toInt,
            s.substring(15, 17).toInt,
            s.substring(18, 20).toInt)
}

def parseApacheLogLine(logline: String): (Either[Row, String], Int) = {
    // Parse a line in the Apache Common Log format
    // Args:
    //    logline (str): a line of text in the Apache Common Log format
    // Returns:
    //    tuple: either a dictionary containing the parts of the Apache Access Log and 1,
    //           or the original invalid log line and 0
    
    val ret = APACHE_ACCESS_LOG_PATTERN.findAllIn(logline).matchData.toList
    if (ret.isEmpty)
        return (Right(logline), 0)

    val r = ret(0)
    val size_field = r.group(9)

    var size: Long = 0
    if (size_field != "-")
        size = size_field.toLong

    return (Left(Row(
            r.group(1),
            r.group(2),
            r.group(3),
            parse_apache_time(r.group(4)),
            r.group(5),
            r.group(6),
            r.group(7),
            r.group(8).toInt,
            size)), 1)
}

def parseLogs(): (RDD[(Either[Row, String], Int)], RDD[Row], RDD[String]) = {
    val fileName = "data/apache/apache.log"
    
    val parsed_logs = sc.textFile(fileName).map(parseApacheLogLine).cache()
    val access_logs = parsed_logs.filter(x => x._2 == 1).map(x => x._1.left.get)
    val failed_logs = parsed_logs.filter(x => x._2 == 0).map(x => x._1.right.get)

    val failed_logs_count = failed_logs.count()
    
    if (failed_logs_count > 0) {
        println(s"Number of invalid logline: $failed_logs.count()")
        failed_logs.take(20).foreach(println)
    }
    
    println(s"Read $parsed_logs.count() lines, successfully parsed $access_logs.count() lines, and failed to parse $failed_logs.count()")
    
    return (parsed_logs, access_logs, failed_logs)
}

// A regular expression pattern to extract fields from the log line
val APACHE_ACCESS_LOG_PATTERN = """^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)""".r

val (parsed_logs, access_logs, failed_logs) = parseLogs()
access_logs.take(20).foreach(println)

Read MapPartitionsRDD[173] at map at <console>:111.count() lines, successfully parsed MapPartitionsRDD[175] at map at <console>:112.count() lines, and failed to parse MapPartitionsRDD[177] at map at <console>:113.count()
Row(in24.inetnebr.com,-,-,Cal(1995,8,1,0,0,1),GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,HTTP/1.0,200,1839)
Row(uplherc.upl.com,-,-,Cal(1995,8,1,0,0,7),GET,/,HTTP/1.0,304,0)
Row(uplherc.upl.com,-,-,Cal(1995,8,1,0,0,8),GET,/images/ksclogo-medium.gif,HTTP/1.0,304,0)
Row(uplherc.upl.com,-,-,Cal(1995,8,1,0,0,8),GET,/images/MOSAIC-logosmall.gif,HTTP/1.0,304,0)
Row(uplherc.upl.com,-,-,Cal(1995,8,1,0,0,8),GET,/images/USA-logosmall.gif,HTTP/1.0,304,0)
Row(ix-esc-ca2-07.ix.netcom.com,-,-,Cal(1995,8,1,0,0,9),GET,/images/launch-logo.gif,HTTP/1.0,200,1713)
Row(uplherc.upl.com,-,-,Cal(1995,8,1,0,0,10),GET,/images/WORLD-logosmall.gif,HTTP/1.0,304,0)
Row(slppp6.intermind.net,-,-,Cal(1995,8,1,0,0,10),GET,/history/skylab/skylab.html,HTTP/1.0,200,1687)
Row(piweba4y.prodigy.com,

### **Part 2: Sample Analyses on the Web Server Log File**
 


Let's compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes. We can compute the statistics by applying a `map` to the `access_logs` RDD. The *lambda* function we want for the map is to extract the `content_size` field from the RDD. The map produces a new RDD containing only the `content_sizes` (one element for each `Row` object in the `access_logs` RDD). To compute the minimum and maximum statistics, we can use `min()` and `max()` functions on the new RDD. We can compute the average statistic by using the `reduce` function with a *lambda* function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the `count()` function on the new RDD.

In [88]:
// TODO: Replace <FILL IN> with appropriate code
// Calculate statistics based on the content size.
val content_sizes = access_logs.map(log => log.content_size).cache()

println("Content Size Avg: " + (content_sizes.reduce(_ + _)/ content_sizes.count()) +
         ", Min: " + content_sizes.min() +
         ", Max: " + content_sizes.max())

Content Size Avg: 17528, Min: 0, Max: 3421948


Next, lets look at the response codes that appear in the log. As with the content size analysis, first we create a new RDD by using a *lambda* function to extract the `response_code` field from the `access_logs` RDD. The difference here is that we will use a "pair tuple" instead of just the field itself. Using a pair tuple consisting of the response code and 1 will let us count how many records have a particular response code. Using the new RDD, we perform a `reduceByKey` function. `reduceByKey` performs a reduce on a per-key basis by applying the *lambda* function to each element, pairwise with the same key. We use the simple *lambda* function of adding the two values. Then, we cache the resulting RDD and create a list by using the `take` function.

In [89]:
// TODO: Replace <FILL IN> with appropriate code
val responseCodeToCount = access_logs.map(log => (log.response_code, 1)).reduceByKey(_ + _).cache()

val responseCodeToCountList = responseCodeToCount.take(100)

println("Found " + responseCodeToCountList.length + " response codes")
println("Response Code Counts: ")
responseCodeToCountList.foreach(x => print(x + " "))

Found 7 response codes
Response Code Counts: 
(404,3137) (200,426183) (302,6994) (304,33660) (500,2) (403,21) (501,3) 

Let's look at hosts that have accessed the server multiple times (e.g., more than ten times). First, we create a new RDD by using a *lambda* function to extract the `host` field from the `access_logs` RDD using a pair tuple consisting of the host and 1, which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a *lambda* function that adds the two values. We then filter the result based on the count of accesses by each host (the second element of each pair) being greater than 10. Next, we extract the host name by performing a `map` with a *lambda* function that returns the first element of each pair. Finally, we extract 20 elements from the resulting RDD.

In [90]:
// TODO: Replace <FILL IN> with appropriate code
// Any hosts that has accessed the server more than 10 times.
val hostCountPairTuple = access_logs.map(log => (log.host, 1))
val hostSum = hostCountPairTuple.reduceByKey(_ + _)
val hostMoreThan10 = hostSum.filter(_._2 > 10)
val hostsPick20 = hostMoreThan10.map(x => x._1).take(20)

println("Any 20 hosts that have accessed more then 10 times: ")
hostsPick20.foreach(x => println(x + " "))

Any 20 hosts that have accessed more then 10 times: 
n1043347.ksc.nasa.gov 
193.74.242.28 
d02.as1.nisiq.net 
jcday.nccts.drenet.dnd.ca 
ip-pdx2-56.teleport.com 
192.112.22.82 
anx3p4.trib.com 
198.77.113.34 
204.235.86.107 
s9.its.bldrdoc.gov 
crc182.cac.washington.edu 
204.255.92.30 
161.243.222.10 
telford-107.salford.ac.uk 
universe6.barint.on.ca 
gatekeeper.homecare.com 
kaifmv.tksc.nasda.go.jp 
unknown.edsa.co.za 
onyx.southwind.net 
sunspot.eds.ecip.nagoya-u.ac.jp 


For the final example, we'll look at the top endpoints (URIs) in the log. To determine them, we first create a new RDD by using a *lambda* function to extract the `endpoint` field from the `access_logs` RDD using a pair tuple consisting of the endpoint and 1 which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a *lambda* function that adds the two values. We then extract the top ten endpoints by performing a `takeOrdered` with a value of 10 and a *lambda* function to create a sorted list with the top endpoints at the bottom.

In [91]:
// TODO: Replace <FILL IN> with appropriate code
// Top Endpoints
val endpointCounts = access_logs.map(log => (log.endpoint, 1)).reduceByKey(_ + _)

val topEndpoints1 = endpointCounts.takeOrdered(10)(Ordering.by[(String,Int),Int](_._2).reverse)
//Another possible choice would be to use the function top for ordering RDDs by value with the top endpoints first.
val topEndpoints = endpointCounts.map(x => (x._2,x._1)).top(10).map(x => (x._2,x._1))

print("Top Ten Endpoints: ")
topEndpoints.foreach(x => println(x + " "))
topEndpoints1.foreach(x => println(x + " "))

Top Ten Endpoints: (/images/NASA-logosmall.gif,27035) 
(/images/KSC-logosmall.gif,21458) 
(/images/MOSAIC-logosmall.gif,20254) 
(/images/USA-logosmall.gif,20172) 
(/images/WORLD-logosmall.gif,20004) 
(/images/ksclogo-medium.gif,19300) 
(/ksc.html,13508) 
(/history/apollo/images/apollo-logo1.gif,11074) 
(/images/launch-logo.gif,10120) 
(/,9481) 
(/images/NASA-logosmall.gif,27035) 
(/images/KSC-logosmall.gif,21458) 
(/images/MOSAIC-logosmall.gif,20254) 
(/images/USA-logosmall.gif,20172) 
(/images/WORLD-logosmall.gif,20004) 
(/images/ksclogo-medium.gif,19300) 
(/ksc.html,13508) 
(/history/apollo/images/apollo-logo1.gif,11074) 
(/images/launch-logo.gif,10120) 
(/,9481) 


### **Part 3: Analyzing Web Server Log File**


What are the top ten endpoints which did not have return code 200? Create a sorted list containing top ten endpoints and the number of times that they were accessed with non-200 return code. Think about the steps that you need to perform to determine which endpoints did not have a 200 return code, how you will uniquely count those endpoints, and sort the list. 

In [92]:
// TODO: Replace <FILL IN> with appropriate code

val not200 = access_logs.filter(log => log.response_code != 200)
val endpointCountPairTuple = not200.map(log => (log.endpoint, 1))
val endpointSum = endpointCountPairTuple.reduceByKey(_ + _)
val topTenErrURLs = endpointSum.takeOrdered(10)(Ordering.by[(String,Int),Int](_._2).reverse)

println("Top Ten failed URLs: ")
topTenErrURLs.foreach(x => println(x + " "))

Top Ten failed URLs: 
(/images/NASA-logosmall.gif,3914) 
(/images/KSC-logosmall.gif,2906) 
(/images/MOSAIC-logosmall.gif,2107) 
(/images/USA-logosmall.gif,2079) 
(/images/WORLD-logosmall.gif,2006) 
(/images/ksclogo-medium.gif,1929) 
(/history/apollo/images/apollo-logo1.gif,1067) 
(/images/launch-logo.gif,1022) 
(/,871) 
(/images/ksclogosmall.gif,663) 


Let's count the number of unique hosts in the entire log. Think about the steps that you need to perform to count the number of different hosts in the log.

In [93]:
// TODO: Replace <FILL IN> with appropriate code

val hosts = access_logs.map(log => log.host)
val uniqueHosts = hosts.distinct()

val uniqueHostCount = uniqueHosts.count()
println("Unique hosts: " + uniqueHostCount)

Unique hosts: 26672


Now, let's determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give us counts of the number of unique daily hosts. We'd like a list sorted by increasing day of the month which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting RDD `dailyHosts` so that we can reuse it in the next exercise. Think about the steps that you need to perform to count the number of different hosts that make requests *each* day. Since the log only covers a single month, you can ignore the month.

In [94]:
// TODO: Replace <FILL IN> with appropriate code

val dayToHostPairTuple = access_logs.map(log => (log.date_time.day,log.host))
val dayGroupedHosts = dayToHostPairTuple.groupByKey()

val dayHostCount = dayGroupedHosts.map(log => (log._1,log._2.toList.distinct.size))
val dailyHosts = dayHostCount.sortByKey(true).cache()
val dailyHostsList = dailyHosts.take(31)

println("Unique hosts per day: ")
dailyHostsList.foreach(x => println(x + " "))

Unique hosts per day: 
(1,2582) 
(3,3222) 
(4,4190) 
(5,2502) 
(6,2537) 
(7,4106) 
(8,4406) 
(9,4317) 
(10,4523) 
(11,2388) 


Next, let's determine the average number of requests on a day-by-day basis. We'd like a list by increasing day of the month and the associated average number of requests per host for that day. Make sure you cache the resulting RDD `avgDailyReqPerHost` so that we can reuse it in the next exercise. To compute the average number of requests per host, get the total number of request across all hosts and divide that by the number of unique hosts. Since the log only covers a single month, you can skip checking for the month. Also to keep it simple, when calculating the approximate average use the integer value.

In [95]:
// TODO: Replace <FILL IN> with appropriate code

val dayAndHostTuple = access_logs.map(log => (log.date_time.day,log.host))
val groupedByDay = dayAndHostTuple.groupByKey()
val sortedByDay = groupedByDay.sortByKey(true)
val avgDailyReqPerHost = sortedByDay.map(log => (log._1,log._2.size/log._2.toList.distinct.size)).cache() 
val avgDailyReqPerHostList = avgDailyReqPerHost.takeOrdered(31)

println("Average number of daily requests per Hosts is ")
avgDailyReqPerHostList.foreach(x => println(x + " "))

Average number of daily requests per Hosts is 
(1,13) 
(3,12) 
(4,14) 
(5,12) 
(6,12) 
(7,13) 
(8,13) 
(9,14) 
(10,13) 
(11,13) 


### **Part 4: Exploring 404 Response Codes**


Let's count the 404 response codes. Create a RDD containing only log records with a 404 response code. Make sure you `cache()` the RDD `badRecords` as we will use it in the rest of this exercise. How many 404 records are in the log?

In [96]:
// TODO: Replace <FILL IN> with appropriate code

val badRecords = access_logs.filter(_.response_code == 404).cache()

println("Found " + badRecords.count() + " 404 URLs.")


Found 3137 404 URLs.


Now, let's list the 404 response code records. Using the RDD containing only log records with a 404 response code that you cached in the previous part, print out a list up to 40 *distinct* endpoints that generate 404 errors -  no endpoint should appear more than once in your list.

In [97]:
// TODO: Replace <FILL IN> with appropriate code

val badEndpoints = badRecords.map(_.endpoint)
val badUniqueEndpoints = badEndpoints.distinct()
val badUniqueEndpointsPick40 = badUniqueEndpoints.take(40)

println("404 URLS: ")
badUniqueEndpointsPick40.foreach(x => println(x + " "))

404 URLS: 
/PERSONS/NASA-CM. 
/shuttle/missions/sts-1/sts-1-mission.html 
/history/apollo/sa-1/sa-1-patch-small.gif 
/public.win3/winvn 
/shuttle/technology/STS_newsref/spacelab.html 
/shutttle/missions/sts-70/images/KSC-95EC-1059.jpg 
/shuttle/missions/sts-70/images/KSC-95EC-o667.gif" 
/%3Aspacelink.msfc.nasa.gov 
/history/apollo/sa-1/images/ 
/:/spacelink.msfc.nasa.gov 
/sts-71/visitor/ 
/history/apollo/sa-10/sa-10-patch-small.gif 
/elv/updated.gif 
/shuttle/missions/mission.html 
/enterprise 
/space/pub/gif 
/netpro/mlm/index.htm 
/shuttle/missions/sts-83/mission-sts-83.html 
/shuttle/technology/missions/missions.html 
/software/winvn/winvn/html 
/news/sci.space.shuttle/archive/sci-space-shuttle-22-apr-1995-40.txt 
/shuttle/missions/sts-69/mission-sts-69.htlm 
/history/gemini/gemini-12.html 
/shuttle/missions/sts-67/images/k95p0383.txt 
/wwwicons/red.gif 
/shuttle/missions/sts-69/mission_sts-69.htlm 
/ksc.shtml 
/history/apollo/a-004/a-004-patch-small.gif 
/shuttle/html 
/www/shuttl

Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top twenty endpoints that generate the most 404 errors. Remember, top endpoints should be in sorted order.

In [98]:
// TODO: Replace <FILL IN> with appropriate code

object ErrOrdering extends Ordering[(String, Int)] {
  def compare(a: (String, Int), b: (String, Int)) = a._2 compare b._2
}

val badEndpointsCountPairTuple = badRecords.map(log => (log.endpoint, 1))
val badEndpointsSum = badEndpointsCountPairTuple.reduceByKey(_ + _)
val badEndpointsTop20 = badEndpointsSum.takeOrdered(20)(ErrOrdering.reverse)

println("Top Twenty 404 URLs: ")
badEndpointsTop20.foreach(x => println(x + " "))

Top Twenty 404 URLs: 
(/images/nasa-logo.gif,319) 
(/pub/winvn/readme.txt,257) 
(/pub/winvn/release.txt,199) 
(/shuttle/missions/STS-69/mission-STS-69.html,181) 
(/elv/DELTA/uncons.htm,106) 
(/images/Nasa-logo.gif,85) 
(/shuttle/missions/sts-68/ksc-upclose.gif,80) 
(/history/apollo/sa-1/sa-1-patch-small.gif,79) 
(/images/crawlerway-logo.gif,63) 
(/://spacelink.msfc.nasa.gov,55) 
(/history/apollo/a-001/a-001-patch-small.gif,49) 
(/shuttle/resources/orbiters/atlantis.gif,39) 
(/history/apollo/images/little-joe.jpg,36) 
(/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,36) 
(/shuttle/countdown/count69.gif,31) 
(/images/lf-logo.gif,27) 
(/history/apollo/sa-5/sa-5-patch-small.gif,24) 
(/shuttle/resources/orbiters/challenger.gif,24) 
(/robots.txt,23) 
(/shuttle/resources/orbiters/discovery.gif,20) 


Instead of looking at the endpoints that generated 404 errors, let's look at the hosts that encountered 404 errors. Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top twenty-five hosts that generate the most 404 errors.

In [99]:
// TODO: Replace <FILL IN> with appropriate code

val errHostsCountPairTuple = badRecords.map(log => (log.host, 1))
val errHostsSum = errHostsCountPairTuple.reduceByKey(_ + _)
val errHostsTop25 = errHostsSum.takeOrdered(25)(Ordering[Int].reverse.on(x => x._2))

println("Top 25 hosts that generated errors: ")
errHostsTop25.foreach(x => println(x + " "))

Top 25 hosts that generated errors: 
(maz3.maz.net,39) 
(nexus.mlckew.edu.au,37) 
(ts8-1.westwood.ts.ucla.edu,37) 
(piweba3y.prodigy.com,34) 
(spica.sci.isas.ac.jp,27) 
(203.13.168.17,25) 
(203.13.168.24,25) 
(www-c4.proxy.aol.com,23) 
(scooter.pa-x.dec.com,23) 
(onramp2-9.onr.com,22) 
(crl5.crl.com,22) 
(198.40.25.102.sap2.artic.edu,21) 
(msp1-16.nas.mr.net,20) 
(gn2.getnet.com,20) 
(isou24.vilspa.esa.es,19) 
(tigger.nashscene.com,19) 
(dial055.mbnet.mb.ca,19) 
(dialup551.chicago.mci.net,18) 
(utsi057.utsi.com,17) 
(cougar.oro.net,17) 
(quadra_alpha.rollins.edu,17) 
(ix-atl10-08.ix.netcom.com,16) 
(micromatix.jagunet.com,16) 
(www-relay.pa-x.dec.com,14) 
(redx3.cac.washington.edu,14) 


Let's explore the 404 records temporally. Break down the 404 requests by day and get the daily counts sorted by day as a list. Since the log only covers a single month, you can ignore the month in your checks.

In [100]:
// TODO: Replace <FILL IN> with appropriate code

val errDateCountPairTuple = badRecords.map(log => (log.date_time.day, 1))
val errDateSum = errDateCountPairTuple.reduceByKey(_ + _)
val errDateSorted = errDateSum.cache()
val errByDate = errDateSorted.takeOrdered(30)

println("404 Errors by day: ")
errByDate.foreach(x => println(x + " "))

404 Errors by day: 
(1,243) 
(3,303) 
(4,346) 
(5,234) 
(6,372) 
(7,532) 
(8,381) 
(9,279) 
(10,314) 
(11,133) 


Using the RDD `errDateSorted` you cached before, what are the top five days for 404 response codes and the corresponding counts of 404 response codes?

In [101]:
// TODO: Replace <FILL IN> with appropriate code

object DateOrdering extends Ordering[(Int, Int)] {
  def compare(a: (Int, Int), b: (Int, Int)) = a._2 compare b._2
}

val topErrDate = errDateSorted.top(5)(DateOrdering)

println("Top Five dates for 404 requests: ")
topErrDate.foreach(x => println(x + " "))

Top Five dates for 404 requests: 
(7,532) 
(8,381) 
(6,372) 
(4,346) 
(10,314) 


Using the RDD `badRecords` you cached before, and by hour of the day and in increasing order, create an RDD containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). Cache the resulting RDD hourRecordsSorted and print that as a list.

In [102]:
// TODO: Replace <FILL IN> with appropriate code

val hourCountPairTuple = badRecords.map(log => (log.date_time.hour, 1))
val hourRecordsSum = hourCountPairTuple.reduceByKey(_ + _)
val hourRecordsSorted = hourRecordsSum.sortByKey(true).cache()
val errHourList = hourRecordsSorted.collect()

println("Top hours for 404 requests: ")
errHourList.foreach(x => println(x + " "))

Top hours for 404 requests: 
(0,101) 
(1,97) 
(2,343) 
(3,197) 
(4,36) 
(5,27) 
(6,45) 
(7,61) 
(8,121) 
(9,91) 
(10,162) 
(11,136) 
(12,239) 
(13,195) 
(14,104) 
(15,164) 
(16,202) 
(17,134) 
(18,130) 
(19,102) 
(20,157) 
(21,105) 
(22,87) 
(23,101) 
